In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

#### This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

### Read the case, department, and source data into their own spark dataframes.

In [2]:
case_df = spark.read.csv("case.csv", header=True, inferSchema=True)

In [3]:
case_df.show(3, False, True)

-RECORD 0-----------------------------------------------------
 case_id              | 1014127332                            
 case_opened_date     | 1/1/18 0:42                           
 case_closed_date     | 1/1/18 12:29                          
 SLA_due_date         | 9/26/20 0:42                          
 case_late            | NO                                    
 num_days_late        | -998.5087616000001                    
 case_closed          | YES                                   
 dept_division        | Field Operations                      
 service_request_type | Stray Animal                          
 SLA_days             | 999.0                                 
 case_status          | Closed                                
 source_id            | svcCRMLS                              
 request_address      | 2315  EL PASO ST, San Antonio, 78207  
 council_district     | 5                                     
-RECORD 1----------------------------------------------

In [4]:
dept_df = spark.read.csv("dept.csv", header=True, inferSchema=True)

In [5]:
dept_df.show(3, False, True)

-RECORD 0----------------------------------------
 dept_division          | 311 Call Center        
 dept_name              | Customer Service       
 standardized_dept_name | Customer Service       
 dept_subject_to_SLA    | YES                    
-RECORD 1----------------------------------------
 dept_division          | Brush                  
 dept_name              | Solid Waste Management 
 standardized_dept_name | Solid Waste            
 dept_subject_to_SLA    | YES                    
-RECORD 2----------------------------------------
 dept_division          | Clean and Green        
 dept_name              | Parks and Recreation   
 standardized_dept_name | Parks & Recreation     
 dept_subject_to_SLA    | YES                    
only showing top 3 rows



In [6]:
source_df = spark.read.csv("source.csv", header=True, inferSchema=True)

In [7]:
source_df.show(3, False, True)

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
-RECORD 1---------------------------
 source_id       | 103582           
 source_username | Carmen Cura      
-RECORD 2---------------------------
 source_id       | 106463           
 source_username | Richard Sanchez  
only showing top 3 rows



In [8]:
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType(
    [
        StructField("source_id", StringType()),
        StructField("source_username", StringType()),
    ]
)


# Read csv, but now we specify the schema:

source = spark.read.csv("source.csv", header=True, schema=schema)

### Let's see how writing to the local disk works in spark:

#### Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

In [9]:
source.write.json("source_json", mode="overwrite")

In [10]:
source.write.csv("source_csv", mode="overwrite")

#### Inspect your folder structure. What do you notice?

In [12]:
# The source_df and source_csv are now part of the folder structure

### Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [13]:
source_df.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



In [14]:
case_df.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [15]:
dept_df.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



#### ###### #####

### 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [17]:
fmt = "M/d/yy H:mm"
case_df = (
    case_df
    .withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_opened_date", fmt))
    .withColumn("SLA_due_date", to_timestamp("SLA_due_date", fmt))
)

In [18]:
open_cases = (
    case_df.filter(expr('case_closed == "NO"'))
    .withColumn('days_past_SLA', datediff(current_timestamp(),'SLA_due_date'))
    .withColumn('duration', datediff(current_timestamp(),'case_opened_date'))
)

In [19]:
# How old is the latest currently opened issue?
open_cases.select(min('days_past_SLA')).show()

+------------------+
|min(days_past_SLA)|
+------------------+
|              -293|
+------------------+



In [20]:
# How long has the oldest issue been open?
open_cases.select(max('duration')).show()

+-------------+
|max(duration)|
+-------------+
|         1679|
+-------------+



### 2. How many Stray Animal cases are there?

In [21]:
# Group by case type
(
    case_df.groupby('service_request_type').count()
    .where(case_df.service_request_type == 'Stray Animal').show()
)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|        Stray Animal|26760|
+--------------------+-----+



### 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [22]:
# 
(
    case_df.select('service_request_type')
    .where(case_df.dept_division == 'Field Operations')
    .where(case_df.service_request_type != 'Officer Standby').count()
)

113902

### 4. Convert the council_district column to a string column.



In [23]:
case_df.groupBy('council_district').count().show(3)

+----------------+------+
|council_district| count|
+----------------+------+
|               1|119309|
|               6| 74095|
|               3|102706|
+----------------+------+
only showing top 3 rows



In [24]:
case_df = case_df.withColumn("council_district", col("council_district").cast("string"))

In [25]:
case_df.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: timestamp (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)
 |-- case_due_date: timestamp (nullable = true)



### 5. Extract the year from the case_closed_date column.

In [26]:
year = case_df.withColumn('year', year('case_closed_date'))

In [27]:
year.select('year').show()

+----+
|year|
+----+
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
+----+
only showing top 20 rows



### 6. Convert num_days_late from days to hours in new columns num_hours_late.


In [29]:
case_df = case_df.withColumn(
    "num_hours_late", expr("num_days_late * 24 AS num_hours_late")
)

case_df.select("num_days_late", "num_hours_late").show(5)

+-------------------+-------------------+
|      num_days_late|     num_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
+-------------------+-------------------+
only showing top 5 rows



### 7. Join the case data with the source and department data.

In [30]:
df = case_df.join(source_df, on=case_df.source_id == source_df.source_id, how = 'left').drop(source_df.source_id)

In [33]:
df = df.join(dept_df, on=df.dept_division == dept_df.dept_division, how = 'left').drop(dept_df.dept_division)

In [34]:
df.show(2, vertical = True)

-RECORD 0--------------------------------------
 case_id                | 1014127332           
 case_opened_date       | 2018-01-01 00:42:00  
 case_closed_date       | 2018-01-01 00:42:00  
 SLA_due_date           | 2020-09-26 00:42:00  
 case_late              | NO                   
 num_days_late          | -998.5087616000001   
 case_closed            | YES                  
 dept_division          | Field Operations     
 service_request_type   | Stray Animal         
 SLA_days               | 999.0                
 case_status            | Closed               
 source_id              | svcCRMLS             
 request_address        | 2315  EL PASO ST,... 
 council_district       | 5                    
 case_due_date          | 2018-01-01 00:42:00  
 num_hours_late         | -23964.2102784       
 source_username        | svcCRMLS             
 dept_name              | Animal Care Services 
 standardized_dept_name | Animal Care Services 
 dept_subject_to_SLA    | YES           

### 8. Are there any cases that do not have a request source?

In [35]:
df.filter("source_username is null").show(vertical=True)

(0 rows)



In [36]:
# no null values

### 9. What are the top 10 service request types in terms of number of requests?

In [37]:
df.groupby('service_request_type').count().sort(col('count').desc()).show(10)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           No Pickup|89210|
|Overgrown Yard/Trash|66403|
|        Bandit Signs|32968|
|        Damaged Cart|31163|
|Front Or Side Yar...|28920|
|        Stray Animal|27361|
|Aggressive Animal...|25492|
|Cart Exchange Req...|22608|
|Junk Vehicle On P...|21649|
|     Pot Hole Repair|20827|
+--------------------+-----+
only showing top 10 rows



### 10. What are the top 10 service request types in terms of average days late?

In [38]:
df.show(1, False, True)

-RECORD 0------------------------------------------------------
 case_id                | 1014127332                           
 case_opened_date       | 2018-01-01 00:42:00                  
 case_closed_date       | 2018-01-01 00:42:00                  
 SLA_due_date           | 2020-09-26 00:42:00                  
 case_late              | NO                                   
 num_days_late          | -998.5087616000001                   
 case_closed            | YES                                  
 dept_division          | Field Operations                     
 service_request_type   | Stray Animal                         
 SLA_days               | 999.0                                
 case_status            | Closed                               
 source_id              | svcCRMLS                             
 request_address        | 2315  EL PASO ST, San Antonio, 78207 
 council_district       | 5                                    
 case_due_date          | 2018-01-01 00:

In [39]:
df.where('case_late=="YES"')\
.groupBy('service_request_type')\
.agg(mean('num_days_late'))\
.sort(desc('avg(num_days_late)'))\
.show(10, truncate=False)

+--------------------------------------+------------------+
|service_request_type                  |avg(num_days_late)|
+--------------------------------------+------------------+
|Zoning: Recycle Yard                  |210.89201994318182|
|Zoning: Junk Yards                    |200.20517608494276|
|Structure/Housing Maintenance         |190.20707698509807|
|Donation Container Enforcement        |171.09115313942615|
|Storage of Used Mattress              |163.96812829714287|
|Labeling for Used Mattress            |162.43032902285717|
|Record Keeping of Used Mattresses     |153.99724039428568|
|Signage Requied for Sale of Used Mattr|151.63868055333333|
|Traffic Signal Graffiti               |137.64583330000002|
|License Requied Used Mattress Sales   |128.79828704142858|
+--------------------------------------+------------------+
only showing top 10 rows



### 11. Does number of days late depend on department?

In [40]:
df.where('case_late=="YES"')\
.groupBy('standardized_dept_name')\
.agg(mean('num_days_late'))\
.sort(desc('avg(num_days_late)'))\
.show(truncate=False)

+------------------------+------------------+
|standardized_dept_name  |avg(num_days_late)|
+------------------------+------------------+
|Customer Service        |88.18248182589822 |
|DSD/Code Enforcement    |49.506339986350454|
|Animal Care Services    |23.4467296347382  |
|Parks & Recreation      |22.427807192724135|
|Trans & Cap Improvements|10.662950455078674|
|Solid Waste             |7.1471727895574135|
|Metro Health            |6.494699602827871 |
+------------------------+------------------+



### 12. How do number of days late depend on department and request type?

In [41]:
df.where('case_late=="YES"')\
.groupBy('standardized_dept_name', 'service_request_type')\
.agg(mean('num_days_late'))\
.sort(desc('avg(num_days_late)'))\
.show(truncate=False)

+------------------------+----------------------------------------+------------------+
|standardized_dept_name  |service_request_type                    |avg(num_days_late)|
+------------------------+----------------------------------------+------------------+
|DSD/Code Enforcement    |Zoning: Recycle Yard                    |210.89201994318182|
|DSD/Code Enforcement    |Zoning: Junk Yards                      |200.20517608494276|
|DSD/Code Enforcement    |Structure/Housing Maintenance           |190.20707698509807|
|DSD/Code Enforcement    |Donation Container Enforcement          |171.09115313942615|
|DSD/Code Enforcement    |Storage of Used Mattress                |163.96812829714287|
|DSD/Code Enforcement    |Labeling for Used Mattress              |162.43032902285717|
|DSD/Code Enforcement    |Record Keeping of Used Mattresses       |153.99724039428568|
|DSD/Code Enforcement    |Signage Requied for Sale of Used Mattr  |151.63868055333333|
|Trans & Cap Improvements|Traffic Signal Gr